In [1]:
import healpy
import numpy as np
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery

from multiorder import AbstractMultiorderMapBuilder
from min_max_mean_state import MinMaxMeanState, MinMaxMeanStateMerger
from paths import *

In [2]:
class StateMerger(MinMaxMeanStateMerger):
    def __init__(self, threshold: float = 0.16):
        self.threshold = threshold
    
    def validate_state(self, state: MinMaxMeanState) -> bool:
        norm = max(map(abs, [state.min, state.max]))
        if norm == 0.0:
            return True
        return (state.max - state.min) / norm <= self.threshold
    

def ebv(norder: int):
    n_size = healpy.order2nside(norder)
    n_pix = healpy.order2npix(norder)
    
    indexes = np.arange(n_pix)
    ra, dec = healpy.pix2ang(n_size, indexes, nest=True, lonlat=True)
    coord = SkyCoord(ra=ra, dec=dec, unit='deg')
    
    sfd_query = SFDQuery(INPUT_DIR)
    return sfd_query(coord)
    

class Builder(StateMerger, AbstractMultiorderMapBuilder):
    def __init__(self, max_norder, threshold):
        AbstractMultiorderMapBuilder.__init__(self, max_norder)
        StateMerger.__init__(self, threshold)
        
        self.ebv = ebv(max_norder)
        
    def calculate_state(self, index_max_norder: int) -> MinMaxMeanState:
        value = self.ebv[index_max_norder].item()
        return MinMaxMeanState(min=value, max=value, mean=value)

In [8]:
%%time

tiles = Builder(max_norder=10, threshold=0.16).build()

CPU times: user 16 s, sys: 617 ms, total: 16.6 s
Wall time: 16.8 s


In [9]:
tiles[6].indexes, tiles[6].values

([6385,
  6707,
  10931,
  12612,
  12623,
  12626,
  12634,
  12653,
  12752,
  12753,
  12758,
  12761,
  13151,
  13482,
  13832,
  13834,
  13856,
  13857,
  13858,
  13859,
  13864,
  13993,
  16394,
  16416,
  16419,
  16912,
  16924,
  16946,
  16953,
  16956,
  16957,
  17000,
  17002,
  17003,
  17006,
  17041,
  17043,
  17044,
  17076,
  17077,
  17079,
  17087,
  17091,
  17098,
  17099,
  17102,
  17106,
  17110,
  17120,
  17121,
  17122,
  17130,
  17144,
  17147,
  17534,
  18501,
  18502,
  18512,
  18513,
  18514,
  18521,
  18523,
  18526,
  18546,
  18552,
  18640,
  18654,
  18677,
  18874,
  19359,
  19432,
  19434,
  19437,
  19438,
  19439,
  24062,
  24666,
  25959,
  25972,
  26506,
  26669,
  26757,
  26772,
  26831,
  26842,
  26855,
  27053,
  27058,
  27070,
  27415,
  28305,
  28308,
  28310,
  28359,
  28381,
  28383,
  28401,
  28554,
  28557,
  28558,
  28559,
  28570,
  28580,
  28582,
  28640,
  28647,
  28649,
  28650,
  28651,
  31362,
  32910,
  3

In [10]:
from mom_builder import mom_from_array

In [11]:
%%time

norder = 6
mom = mom_from_array(ebv(norder).astype(np.float64), norder, 0.16)

CPU times: user 32 ms, sys: 45.7 ms, total: 77.7 ms
Wall time: 265 ms


In [7]:
mom

[(array([], dtype=uint64), array([], dtype=float64)),
 (array([], dtype=uint64), array([], dtype=float64)),
 (array([], dtype=uint64), array([], dtype=float64)),
 (array([], dtype=uint64), array([], dtype=float64)),
 (array([], dtype=uint64), array([], dtype=float64)),
 (array([   43,    69,    70, ..., 12262, 12274, 12282], dtype=uint64),
  array([0.09498527, 0.3297796 , 0.37662794, ..., 0.05478248, 0.05867209,
         0.05462224])),
 (array([    0,     1,     2, ..., 49149, 49150, 49151], dtype=uint64),
  array([0.09396797, 0.09965985, 0.08228148, ..., 0.08659782, 0.09656756,
         0.07763848]))]